<a href="https://colab.research.google.com/github/renatamontanhana/SPARK_Sistema_de_Recomendacao/blob/main/SPARK_Sistema_de_Recomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='purple'>Ciência dos Dados</font>
## <font color='blue'>BigData com Spark</font>

## <font color='blue'>Sistema de Recomendacao</font>

### *********** Atenção: ***********
Utilize Java JDK 1.8 ou 11 e Apache Spark 2.4.2

****** Caso receba mensagem de erro "name 'sc' is not defined", interrompa o pyspark e apague o diretório metastore_db no mesmo diretório onde está este Jupyter notebook ******

## <font color='blue'>Spark MLLib - Sistema de Recomendação</font>

<strong> Descrição </strong>
<ul style="list-style-type:square">
  <li>Também chamado de filtros colaborativos.</li>
  <li>Analisa dados passados para compreender comportamentos de pessoas/entidades.</li>
  <li>A recomendação é feita por similaridade de comportamento.</li>
  <li>Recomendação baseada em usuários ou items.</li>
  <li>Algoritmos de Recomendação esperam receber os dados em um formato específico: [user_ID, item_ID, score].</li>
  <li>Score, também chamado rating, indica a preferência de um usuário sobre um item. Podem ser valores booleanos, ratings ou mesmo volume de vendas.</li>
</ul>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=4c978f87a75b9188b48fc578091e4048d0b4463661ad3e68bbd012e99616c38d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
# Imports de pacotes
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.sql import SparkSession

# Criar uma SparkSession
spSession = SparkSession.builder.master("local").appName("DSA-SparkMLLib").getOrCreate()


In [5]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder.master("local").appName("DSA-SparkMLLib").getOrCreate()

In [11]:
from pyspark.sql import SparkSession

# Obtém ou cria uma SparkSession
spSession = SparkSession.builder.appName("DSA-SparkMLLib").getOrCreate()

# Obtém o SparkContext
sc = spSession.sparkContext

# Agora você pode carregar os dados
ratingsRDD = sc.textFile("/content/user-item.txt")
ratingsRDD.collect()


['1001,9001,10',
 '1001,9002,1',
 '1001,9003,9',
 '1002,9001,3',
 '1002,9002,5',
 '1002,9003,1',
 '1002,9004,10',
 '1003,9001,2',
 '1003,9002,6',
 '1003,9003,2',
 '1003,9004,9',
 '1003,9005,10',
 '1003,9006,8',
 '1003,9007,9',
 '1004,9001,9',
 '1004,9002,2',
 '1004,9003,8',
 '1004,9004,3',
 '1004,9010,10',
 '1004,9011,9',
 '1004,9012,8',
 '1005,9001,8',
 '1005,9002,3',
 '1005,9003,7',
 '1005,9004,1',
 '1005,9010,9',
 '1005,9011,10',
 '1005,9012,9',
 '1005,9013,8',
 '1005,9014,1',
 '1005,9015,1',
 '1006,9001,7',
 '1006,9002,4',
 '1006,9003,8',
 '1006,9004,1',
 '1006,9010,7',
 '1006,9011,6',
 '1006,9012,9']

In [12]:
# Carrega os dados no formato ALS (user, item, rating)
ratingsRDD = sc.textFile("/content/user-item.txt")
ratingsRDD.collect()

['1001,9001,10',
 '1001,9002,1',
 '1001,9003,9',
 '1002,9001,3',
 '1002,9002,5',
 '1002,9003,1',
 '1002,9004,10',
 '1003,9001,2',
 '1003,9002,6',
 '1003,9003,2',
 '1003,9004,9',
 '1003,9005,10',
 '1003,9006,8',
 '1003,9007,9',
 '1004,9001,9',
 '1004,9002,2',
 '1004,9003,8',
 '1004,9004,3',
 '1004,9010,10',
 '1004,9011,9',
 '1004,9012,8',
 '1005,9001,8',
 '1005,9002,3',
 '1005,9003,7',
 '1005,9004,1',
 '1005,9010,9',
 '1005,9011,10',
 '1005,9012,9',
 '1005,9013,8',
 '1005,9014,1',
 '1005,9015,1',
 '1006,9001,7',
 '1006,9002,4',
 '1006,9003,8',
 '1006,9004,1',
 '1006,9010,7',
 '1006,9011,6',
 '1006,9012,9']

In [13]:
# Convertendo as strings
ratingsRDD2 = ratingsRDD.map(lambda l: l.split(',')).map(lambda l:(int(l[0]), int(l[1]), float(l[2])))

In [14]:
# Criando um Dataframe
ratingsDF = spSession.createDataFrame(ratingsRDD2, ["user", "item", "rating"])

In [15]:
ratingsDF

DataFrame[user: bigint, item: bigint, rating: double]

In [16]:
# Construindo o modelo
# ALS = Alternating Least Squares --> Algoritmo para sistema de recomendação, que otimiza a loss function
# e funciona muito bem em ambientes paralelizados
als = ALS(rank = 10, maxIter = 5)
modelo = als.fit(ratingsDF)

In [17]:
# Visualizando o Affinity Score
modelo.userFactors.orderBy("id").collect()

[Row(id=1001, features=[-0.6930740475654602, -0.3868650197982788, 0.41757887601852417, 0.31486400961875916, 0.6179812550544739, 0.6426745653152466, -0.4178941249847412, 0.9820318818092346, 0.09449205547571182, 0.5916354656219482]),
 Row(id=1002, features=[0.5523760914802551, 0.016237694770097733, 0.29231345653533936, -1.1382346153259277, -0.3404906094074249, 0.6859951019287109, -0.08520103991031647, -0.1842235028743744, 1.1725407838821411, 0.15252146124839783]),
 Row(id=1003, features=[0.7773934006690979, 0.11161402612924576, 0.25314968824386597, -0.9876330494880676, 0.11273720115423203, 0.31789934635162354, -0.07186150550842285, 0.018017131835222244, 0.9068472981452942, -0.4528549909591675]),
 Row(id=1004, features=[-0.6240087151527405, -0.3842594027519226, 0.48865941166877747, -0.08379887044429779, 0.48245587944984436, 0.8785684704780579, -0.4429779648780823, 0.5047745108604431, 0.49951472878456116, 0.34139978885650635]),
 Row(id=1005, features=[0.18583036959171295, -0.09427300095558

In [18]:
# Criando um dataset de teste com usuários e items para rating
testeDF = spSession.createDataFrame([(1001,9004),(1001,9005),(1001,9010)], ["user", "item"])

In [19]:
# Previsões
# Quanto maior o Affinity Score, maior a probabilidade do usuário aceitar uma recomendação
previsoes = (modelo.transform(testeDF).collect())
previsoes

[Row(user=1001, item=9004, prediction=-0.617450475692749),
 Row(user=1001, item=9005, prediction=-2.532466411590576),
 Row(user=1001, item=9010, prediction=11.25540542602539)]

# Fim

### Tamojunto - Ciencia dos Dados <a href="http://facebook.com/cienciadosdadosbr">facebook.com/cienciadosdadosbr</a>

### #**Telegram** - Scripts e Datasets - Comunidade Telegram <a href="https://t.me/cienciadosdados">https://t.me/cienciadosdadosraiz</a>

### #YouTube - Mais Aulas como essa no YouTube <a href="https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s">https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s</a>